In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import gc
import glob

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [4]:
Q3_start="2019-08-04"
Q3_end="2019-11-02"

daily_item_files_2019Q3=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019Q3=[x for x in daily_item_files_2019Q3 if x[-4:]==".txt" and "daily" in x.lower()]
daily_item_files_2019Q3=[x for x in daily_item_files_2019Q3 if x.split("s/MediaStorm_")[1][:10]>=Q3_start]
daily_item_files_2019Q3=[x for x in daily_item_files_2019Q3 if x.split("s/MediaStorm_")[1][:10]<=Q3_end]

print(len(daily_item_files_2019Q3))

13


In [5]:
df_output_Q3_sales=pd.DataFrame()
for file in daily_item_files_2019Q3:
    week_end_dt=file.split("s/MediaStorm_")[1][:10]
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt', 'transaction_id',
                                                              'customer_id_hashed', 'item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    # Rewards
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_rewards_sales=df_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_rewards_sales=df_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt', 'transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['trans']=1
    df_rewards_trans=df_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()
    
    df_rewards=pd.merge(df_rewards_trans,df_rewards_sales,on="location_id",how="outer")
    df_rewards['rewards_label']="rewards"
    
    # Non Rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_non_rewards_sales=df_non_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt', 'transaction_id']].drop_duplicates()
    df_non_rewards_trans['trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()  
    
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on="location_id",how="outer")
    df_non_rewards['rewards_label']="non_rewards"
    
    df=df_rewards.append(df_non_rewards)
    df['store_type']=np.where(df['location_id']=="6990","online","instre")
    df=df.groupby(["rewards_label","store_type"])['sales','trans'].sum().reset_index()
    df['week_end_dt']=week_end_dt
    
    df_output_Q3_sales=df_output_Q3_sales.append(df)
    print(datetime.datetime.now(),file)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2019-11-06 13:58:12.771176 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-10/MediaStormDailySales20190813-120541-143.txt
2019-11-06 13:58:49.933403 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-17/MediaStormDailySales20190820-113628-440.txt
2019-11-06 13:59:30.611830 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-24/MediaStormDailySales20190827-113630-114.txt
2019-11-06 14:00:10.219967 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-08-31/MediaStormDailySales20190903-113649-462.txt
2019-11-06 14:00:39.481498 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-07/MediaStormDailySales20190910-120643-163.txt
2019-11-06 14:01:16.219493 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-14/MediaStormDailySales20190917-110425-322.txt
2019-11-06 14:01:52.254957 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-21/MediaStormDailySales20190924-112126-506.txt
2019-11-06 14:02:36.218309 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-28/MediaStormDailySales20191001-111241-

In [6]:
writer=pd.ExcelWriter("./BL_Q3_total_sales_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_Q3_sales.to_excel(writer,"Q3_total_sales_12_weeks",index=False)
writer.save()

In [7]:
df_output_Q3_sales.shape

(52, 5)

In [8]:
df_non_rewards_sales['sales'].sum()

33453214.960000075

In [9]:
df_non_rewards_trans['trans'].sum()

1255311